<a href="https://colab.research.google.com/github/Mashedpotatoagain/YouBible_Scapper/blob/main/Ao_scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
from bs4 import BeautifulSoup
import requests
import time


verse_urls = [
    {"url":"https://www.bible.com/bible/406/{}.{}.EASY", "lang":"english"},
    {"url":"https://www.bible.com/bible/3663/{}.{}.ANSEBSI", "lang": "ao"}
]

In [21]:
# each book
books = [
    "GEN",
    "EXO",
]

chapters = [
    50,
    40,
]

In [22]:
def extract_verses(soup):

  verses=[]
  previous_verse= 1
  content_verses = soup.find_all("span", class_="ChapterContent_verse__57FIw")

  for content_verse in content_verses:
      verse_label=content_verse.find("span", class_="ChapterContent_label__R2PLt")
      verse_number=""
      if verse_label is not None:
        verse_number=verse_label.get_text(strip=True)
        try:
          previous_verse=int(verse_number)
        except:
          verse_number=""

      verse=content_verse.find_all("span", class_="ChapterContent_content__RrUqA")
      line= [verse_line.get_text(strip=True, separator=' ') for verse_line in verse]
      verse_text = " ".join(line)

      if verse_number=="":
        prev_verse = verses[previous_verse-1]
        prev_verse["text"]+= verse_text.strip()
        verses[previous_verse-1] = prev_verse

      else:
        verses.append({"no": verse_number, "text": verse_text.strip()})
  return verses



In [23]:
def extract_verses(soup):
  verses = []
  previous_verse = 0

  for verse in soup.find_all("span", attrs={"class": "verse"}):
    verse_number = verse.find("a", attrs={"class": "verse-number"})
    if verse_number is not None:
      verse_number = verse_number.text
    else:
      verse_number = ""

    verse_text = verse.find("span", attrs={"class": "verse-text"})
    if verse_text is not None:
      verse_text = verse_text.text

    if verse_number == "":
      prev_verse = verses[previous_verse - 1]
      prev_verse["text"] += verse_text.strip()
      verses[previous_verse - 1] = prev_verse
    else:
      verses.append({"number": verse_number, "text": verse_text})
      previous_verse += 1

  return verses

In [24]:
def scrapeChapter(book, chapter):
  soup = []
  results = {
    "book": book,
    "chapter": chapter,
  }

  for idx, chapterInfo in enumerate(verse_urls):
      page_to_scrape = requests.get(chapterInfo["url"].format(book, chapter))
      soup.append(BeautifulSoup(page_to_scrape.text, "html.parser"))

  lang_1_verses = extract_verses(soup[0])
  lang_2_verses = extract_verses(soup[1])

  results[verse_urls[0]["lang"]] = lang_1_verses
  results[verse_urls[1]["lang"]] = lang_2_verses


  return results

In [25]:
# wait for 2 minutes after scraping 5 chapters

results = []
for book_index in range(len(books)):
  book = books[book_index]
  max_chapter = chapters[book_index]

  # for demo, adding limit to limit reading 2 chapters per book
  limit = 2

  for chapter in range(1, max_chapter+1):
    # implement the waiting/delaying between each chapter reading. ie. for every 5 chapters read, etc.
    # can remove these 2 lines if you don't want limit
    if limit > 2:
      break
    results.append(scrapeChapter(book, str(chapter)))

In [26]:
# to get the output txt, run this

english_lines = []
ao_lines = []

for result in results:

  for verse_index in range(len(result["english"])):

    verse_en = result["english"][verse_index]
    english_lines.append(verse_en["text"]+ "\n")

    verse_ao = result["ao"][verse_index]
    ao_lines.append(verse_ao["text"]+"\n")


with open("english.txt", "w") as f:
  f.writelines(english_lines)

  f.close()


with open("ao.txt", "w") as f:
  f.writelines(ao_lines)

  f.close()






